# Introduction
* In many projects it is hard to find the desired data in accessible and orderly files, and it is necessary to import the data yourself from the Internet using web scraping.
* In this mini-project, I wanted to learn and practice web scraping in general and specifically the BeautifulSoup library.
* I scraped several pages that contain NBA stats data and organized it as a labeled training dataset.

# Import

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import re
import pickle

# Config

In [2]:
year = 2019
mainUrl = 'https://www.basketball-reference.com'
suffixes = ['totals', 'per_game', 'per_minute', 'per_poss', 'advanced']
urlDict = {suff:f"/leagues/NBA_{year}_{suff}.html" for suff in suffixes}

urlDict

{'totals': '/leagues/NBA_2019_totals.html',
 'per_game': '/leagues/NBA_2019_per_game.html',
 'per_minute': '/leagues/NBA_2019_per_minute.html',
 'per_poss': '/leagues/NBA_2019_per_poss.html',
 'advanced': '/leagues/NBA_2019_advanced.html'}

# Functions

In [3]:
def get_data_from_soup_table_with_merged_cells(table):

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text

        data.append(rowD)
        
    return data

# Web scraping

## Get the stats per player from the different pages

In [4]:
soupDict = {}

for name, subUrl in urlDict.items():
    url = mainUrl + subUrl
    html = urlopen(url)
    soup = BeautifulSoup(html)
    soupDict[name] = soup

In [5]:
statsDict = {}

for name, soup in soupDict.items():
    
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')][1:]
    
    rows = soup.findAll('tr')[1:]
    playerStats = [[td.getText() for td in rows[i].findAll('td')] + \
                    [rows[i].find('a', attrs={'href': re.compile("/players")}),
                     rows[i].find('a', attrs={'href': re.compile("/teams")})] \
                    for i in range(len(rows))]

    stats = pd.DataFrame(playerStats, columns=headers+['player_link', 'team_link'])
    stats = stats.loc[~pd.isna(stats['Player'])]
    
    stats['player_link'] = stats['player_link'].apply(lambda bs_link: mainUrl+bs_link.get('href') if bs_link else None)
    stats['team_link'] = stats['team_link'].apply(lambda bs_link: mainUrl+bs_link.get('href') if bs_link else None)
    
    statsDict[name] = stats

In [6]:
for page, df in statsDict.items():
    print(f"Page: {page}")
    display(df.head(3))
    print(f"Shape: {df.shape}\n")

Page: totals


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,player_link,team_link
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,.357,...,43,48,20,17,6,14,53,165,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/OKC...
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,.222,...,22,25,8,1,4,4,24,17,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/PHO...
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,.345,...,49,60,65,14,5,28,45,108,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/ATL...


Shape: (708, 31)

Page: per_game


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,player_link,team_link
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/OKC...
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/PHO...
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/ATL...


Shape: (708, 31)

Page: per_minute


...

Shape: (708, 30)

Page: per_poss


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,,ORtg,DRtg,player_link,team_link
0,Álex Abrines,SG,25,OKC,31,2,588,4.4,12.5,.357,...,1.3,0.5,1.1,4.2,13.1,,103,111,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/OKC...
1,Quincy Acy,PF,28,PHO,10,0,123,1.6,7.0,.222,...,0.4,1.6,1.6,9.3,6.6,,87,116,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/PHO...
2,Jaylen Adams,PG,22,ATL,34,1,428,4.1,11.9,.345,...,1.5,0.5,3.0,4.9,11.7,,99,115,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/ATL...


Shape: (708, 33)

Page: advanced


,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,player_link,team_link
0,Álex Abrines,SG,25,OKC,31,588,6.3,.507,.809,.083,...,0.6,0.6,.053,,-3.7,0.4,-3.3,-0.2,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/OKC...
1,Quincy Acy,PF,28,PHO,10,123,2.9,.379,.833,.556,...,0.0,-0.1,-0.022,,-7.6,-0.5,-8.1,-0.2,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/PHO...
2,Jaylen Adams,PG,22,ATL,34,428,7.6,.474,.673,.082,...,0.2,0.1,.011,,-3.8,-0.5,-4.3,-0.2,https://www.basketball-reference.com/players/a...,https://www.basketball-reference.com/teams/ATL...


Shape: (708, 30)



## Merge all the pages into a single dataframe

In [7]:
columnsToMergeBy = ['Player','Pos', 'Age', 'Tm', 'G', 'GS', 'player_link', 'team_link'] 

for i, (name, df) in enumerate(statsDict.items()):
    dfTemp = df.copy()
    dfTemp.columns = [f"{column}_{name}" \
                      if column not in columnsToMergeBy else column 
                      for column in df.columns]
    if i == 0:
        fullDf = dfTemp
    else:
        columnsToMergeByTemp = [col for col in columnsToMergeBy if col in dfTemp.columns]
        fullDf = fullDf.merge(dfTemp, on=columnsToMergeByTemp, how='outer')

In [8]:
fullDf.head(3)

,Player,Pos,Age,Tm,G,GS,MP_totals,FG_totals,FGA_totals,FG%_totals,...,_advanced,OWS_advanced,DWS_advanced,WS_advanced,WS/48_advanced,_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,.357,...,,0.1,0.6,0.6,.053,,-3.7,0.4,-3.3,-0.2
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,.222,...,,-0.1,0.0,-0.1,-0.022,,-7.6,-0.5,-8.1,-0.2
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,.345,...,,-0.1,0.2,0.1,.011,,-3.8,-0.5,-4.3,-0.2


## Get the "+/-" mertics which will use as labels from each team page

In [9]:
urlPerTeamDf = fullDf[~pd.isna(fullDf['team_link'])][['Tm','team_link']].drop_duplicates()
urlPerTeamDict = {tm:url for tm,url in zip(urlPerTeamDf['Tm'],urlPerTeamDf['team_link'])}

In [10]:
dfPerTeamDict = {}

for team, url in urlPerTeamDict.items():

    html = urlopen(url)
    soup = BeautifulSoup(html)

    for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
        if (comment.find("<table ") > 0) & (comment.find('id="pbp"') > 0):
            commentSoup = BeautifulSoup(comment, 'html.parser')
            table = commentSoup.find("table")

    data = get_data_from_soup_table_with_merged_cells(table)
    df = pd.DataFrame(data[2:])
    columnsName = [f"{a}_{b}".replace('\xa0','') for a, b in zip(data[0], data[1])]
    columnsName = ['Player' if col == '_' else (col[1:] if col.startswith('_') else col) for col in columnsName]
    df.columns = columnsName
    df = df.loc[~pd.isna(df['Player'])]
    
    dfPerTeamDict[team] = df

## Merge all into a single dataframe

In [11]:
for i, (team, df) in enumerate(dfPerTeamDict.items()):
    df['Tm'] = team
    if i == 0:
        fullTeamDf = df
    else:
        fullTeamDf = pd.concat([fullTeamDf, df])

In [12]:
columnsToSelect = ['Tm', 'Player', '+/- Per 100 Poss._OnCourt', '+/- Per 100 Poss._On-Off']
fullDfWithLabels = fullDf.merge(fullTeamDf[columnsToSelect],
                               on=['Tm','Player'], how='inner')

In [13]:
# Remove the sign "+" so the columns could be trasformed to float type.
for column in ['+/- Per 100 Poss._OnCourt', '+/- Per 100 Poss._On-Off']:
    fullDfWithLabels[column] = fullDfWithLabels[column].apply(lambda x: x.replace('+',''))

In [14]:
fullDfWithLabels.head(3)

,Player,Pos,Age,Tm,G,GS,MP_totals,FG_totals,FGA_totals,FG%_totals,...,DWS_advanced,WS_advanced,WS/48_advanced,_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced,+/- Per 100 Poss._OnCourt,+/- Per 100 Poss._On-Off
0,Álex Abrines,SG,25,OKC,31,2,588,56,157,.357,...,0.6,0.6,.053,,-3.7,0.4,-3.3,-0.2,1.6,-1.7
1,Quincy Acy,PF,28,PHO,10,0,123,4,18,.222,...,0.0,-0.1,-0.022,,-7.6,-0.5,-8.1,-0.2,1.5,10.8
2,Jaylen Adams,PG,22,ATL,34,1,428,38,110,.345,...,0.2,0.1,.011,,-3.8,-0.5,-4.3,-0.2,-4.9,1.1


In [15]:
pickle_out = open(f"fullDfWithLabels.pkl","wb")
pickle.dump(fullDfWithLabels, pickle_out)
pickle_out.close()